In [1]:
import pandas as pd
import numpy as np
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt

In [2]:
audio_files='./Music_Data/genres_original'
file_data=pd.read_csv('./Music_Data/features_30_sec.csv')
file_data.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.wav,661794,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,...,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035,blues
1,blues.00001.wav,661794,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,...,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282,blues
2,blues.00002.wav,661794,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,...,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025,blues
3,blues.00003.wav,661794,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,...,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339,blues
4,blues.00004.wav,661794,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,...,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160,blues


In [3]:
extracted_features=[]
for index_num,row in file_data.iterrows():
    try:
        final_class_labels=row["label"]
        file_name = os.path.join(os.path.abspath(audio_files), final_class_labels+'/',str(row["filename"]))    
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        scaled_features = np.mean(features.T,axis=0)
        extracted_features.append([scaled_features,final_class_labels])
    except Exception as e:
        print(f"Error: {e}")
        continue

/home/ben/miniconda3/envs/Flask_env/lib/python3.9/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


Error: 


In [4]:
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','genre'])
extracted_features_df.head()

,feature,genre
0,"[-113.59882, 121.57067, -19.162262, 42.363937,...",blues
1,"[-207.52383, 123.98514, 8.94702, 35.86715, 2.9...",blues
2,"[-90.757164, 140.44087, -29.084547, 31.686693,...",blues
3,"[-199.57513, 150.0861, 5.663404, 26.855282, 1....",blues
4,"[-160.35417, 126.209496, -35.581394, 22.139256...",blues


In [5]:
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['genre'].tolist())


In [6]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

In [7]:
y.shape

(999, 10)

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.18,random_state=0)

In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics


model = keras.Sequential([

        # input layer
        keras.layers.Dense(512,input_shape=(40,), activation='relu'),

        # 1st dense layer
        keras.layers.Dense(256, activation='relu'),

        # 2nd dense layer
        keras.layers.Dense(64, activation='relu'),

        # output layer
        keras.layers.Dense(10, activation='softmax')
    ])

In [10]:
    optimiser = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=optimiser,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               20992     
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 64)                16448     
                                                                 
 dense_3 (Dense)             (None, 10)                650       
                                                                 
Total params: 169,418
Trainable params: 169,418
Non-trainable params: 0
_________________________________________________________________


In [12]:
print(X_train.shape)
print(y_train.shape)


(819, 40)
(819, 10)


In [15]:
    # train model
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=32, epochs=50)

Epoch 1/50
26/26 [==============================] - 0s 8ms/step - loss: 0.4057 - accuracy: 0.8999 - val_loss: 1.1074 - val_accuracy: 0.6333
Epoch 2/50
26/26 [==============================] - 0s 7ms/step - loss: 0.4018 - accuracy: 0.9048 - val_loss: 1.0807 - val_accuracy: 0.6167
Epoch 3/50
26/26 [==============================] - 0s 7ms/step - loss: 0.3961 - accuracy: 0.9072 - val_loss: 1.1637 - val_accuracy: 0.6056
Epoch 4/50
26/26 [==============================] - 0s 8ms/step - loss: 0.3969 - accuracy: 0.8999 - val_loss: 1.1422 - val_accuracy: 0.6056
Epoch 5/50
26/26 [==============================] - 0s 7ms/step - loss: 0.3970 - accuracy: 0.8950 - val_loss: 1.1693 - val_accuracy: 0.5944
Epoch 6/50
26/26 [==============================] - 0s 7ms/step - loss: 0.3758 - accuracy: 0.9133 - val_loss: 1.0980 - val_accuracy: 0.6167
Epoch 7/50
26/26 [==============================] - 0s 5ms/step - loss: 0.3721 - accuracy: 0.9121 - val_loss: 1.1075 - val_accuracy: 0.6333
Epoch 8/50
26/26 [==

In [16]:
test_loss, test_acc = model.evaluate(X_test, y_test, batch_size=128)
print("Test loss is: ", test_loss)
print("The best accuracy is: ", test_acc*100)

2/2 [==============================] - 0s 6ms/step - loss: 1.1835 - accuracy: 0.6500
Test loss is:  1.1835347414016724
The best accuracy is:  64.99999761581421


64.999

In [17]:
model.save('./music_model/model.h5')

In [20]:
import soundfile as sf
import io
classes = {0:'blues', 1:'classical', 2:'country', 3:'disco', 4:'hiphop',
                5:'jazz', 6:'metal', 7:'pop', 8:'reggae', 9:'rock'}

#filename="./Trivium - Rise Above The Tides (Audio).wav"
new_model = load_model('./music_model/model.h5')
with io.open(filename, 'rb') as song_file:
        audio, sample_rate = librosa.load(song_file, res_type='kaiser_fast') 
        mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
        mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)
        predicted_value=model.predict(mfccs_scaled_features)
        predicted_label=np.argmax(predicted_value,axis=1)
        prediction_genre = classes[predicted_label[0]]
print(prediction_genre)

FileNotFoundError: [Errno 2] No such file or directory: './Trivium - Rise Above The Tides (Audio).wav'